# TFM - Análisis de Dataset Público

**Autor:** [Tu nombre]

**Fecha:** [Fecha]

**Dataset:** [Nombre del dataset]

---

## Tabla de Contenidos

1. [Introducción](#1-introducción)
2. [Carga y Exploración de Datos](#2-carga-y-exploración-de-datos)
3. [Análisis Descriptivo](#3-análisis-descriptivo)
4. [Transformación de Datos](#4-transformación-de-datos)
5. [Modelización Predictiva](#5-modelización-predictiva)
6. [Evaluación de Modelos](#6-evaluación-de-modelos)
7. [Discusión de Resultados](#7-discusión-de-resultados)
8. [Conclusiones](#8-conclusiones)

## 1. Introducción

### 1.1 Descripción del Problema

[Describe el problema que se va a abordar y el contexto del dataset]

### 1.2 Objetivos

- Objetivo principal: [Define el objetivo principal del análisis]
- Objetivos específicos:
  - [Objetivo 1]
  - [Objetivo 2]
  - [Objetivo 3]

### 1.3 Dataset

- **Fuente:** [URL o referencia del dataset]
- **Descripción:** [Breve descripción del dataset]
- **Número de instancias:** [N]
- **Número de características:** [M]
- **Variable objetivo:** [Variable de interés]

In [ ]:
# Importación de librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

# Configuración
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

# Semilla para reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 2. Carga y Exploración de Datos

### 2.1 Carga del Dataset

In [ ]:
# Cargar datos
# NOTA: Reemplaza 'your_dataset.csv' con el nombre real de tu archivo
# df = pd.read_csv('../data/raw/your_dataset.csv')

# Ejemplo con dataset de prueba (Iris)
from sklearn.datasets import load_iris
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

print(f"Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")

### 2.2 Primeras Observaciones

In [ ]:
# Primeras filas
display(df.head())

# Información general
print("\nInformación del dataset:")
df.info()

# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
display(df.describe())

### 2.3 Calidad de Datos

In [ ]:
# Valores nulos
print("Valores nulos por columna:")
null_counts = df.isnull().sum()
null_percentages = (null_counts / len(df)) * 100
null_df = pd.DataFrame({
    'Nulos': null_counts,
    'Porcentaje': null_percentages
})
display(null_df[null_df['Nulos'] > 0])

# Duplicados
duplicates = df.duplicated().sum()
print(f"\nFilas duplicadas: {duplicates}")

## 3. Análisis Descriptivo

### 3.1 Análisis Univariado

In [ ]:
# Distribución de variables numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

fig, axes = plt.subplots(len(numeric_cols), 2, figsize=(15, 4*len(numeric_cols)))

for idx, col in enumerate(numeric_cols):
    # Histograma
    axes[idx, 0].hist(df[col].dropna(), bins=30, edgecolor='black', alpha=0.7)
    axes[idx, 0].set_title(f'Distribución de {col}')
    axes[idx, 0].set_xlabel(col)
    axes[idx, 0].set_ylabel('Frecuencia')
    
    # Boxplot
    axes[idx, 1].boxplot(df[col].dropna())
    axes[idx, 1].set_title(f'Boxplot de {col}')
    axes[idx, 1].set_ylabel(col)

plt.tight_layout()
plt.savefig('../results/visualizations/univariate_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

### 3.2 Análisis de la Variable Objetivo

In [ ]:
# Distribución de la variable objetivo
target_col = 'target'  # Reemplaza con tu variable objetivo

if target_col in df.columns:
    plt.figure(figsize=(10, 5))
    
    if df[target_col].dtype in ['int64', 'object']:
        # Variable categórica
        value_counts = df[target_col].value_counts()
        plt.subplot(1, 2, 1)
        value_counts.plot(kind='bar')
        plt.title(f'Distribución de {target_col}')
        plt.xlabel(target_col)
        plt.ylabel('Frecuencia')
        
        plt.subplot(1, 2, 2)
        plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%')
        plt.title(f'Proporción de {target_col}')
    else:
        # Variable continua
        plt.subplot(1, 2, 1)
        plt.hist(df[target_col], bins=30, edgecolor='black')
        plt.title(f'Distribución de {target_col}')
        plt.xlabel(target_col)
        
        plt.subplot(1, 2, 2)
        plt.boxplot(df[target_col])
        plt.title(f'Boxplot de {target_col}')
    
    plt.tight_layout()
    plt.savefig('../results/visualizations/target_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

### 3.3 Análisis Multivariado

In [ ]:
# Matriz de correlación
plt.figure(figsize=(12, 10))
correlation_matrix = df[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación')
plt.tight_layout()
plt.savefig('../results/visualizations/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Identificar correlaciones fuertes
print("\nCorrelaciones más fuertes (|r| > 0.7):")
high_corr = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            high_corr.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

if high_corr:
    for var1, var2, corr in high_corr:
        print(f"{var1} - {var2}: {corr:.3f}")
else:
    print("No se encontraron correlaciones fuertes")

In [ ]:
# Pairplot (para datasets pequeños)
if len(numeric_cols) <= 6 and len(df) <= 1000:
    if target_col in df.columns:
        sns.pairplot(df, hue=target_col, diag_kind='kde', corner=True)
        plt.savefig('../results/visualizations/pairplot.png', dpi=300, bbox_inches='tight')
        plt.show()
else:
    print("Dataset demasiado grande para pairplot completo. Selecciona variables específicas.")

## 4. Transformación de Datos

### 4.1 Tratamiento de Valores Nulos

In [ ]:
# Crear copia para transformaciones
df_clean = df.copy()

# Estrategias de imputación
# Ejemplo: imputar con la mediana para variables numéricas
numeric_cols_with_nulls = df_clean[numeric_cols].columns[df_clean[numeric_cols].isnull().any()].tolist()

for col in numeric_cols_with_nulls:
    median_value = df_clean[col].median()
    df_clean[col].fillna(median_value, inplace=True)
    print(f"Columna '{col}': imputada con mediana ({median_value:.2f})")

print(f"\nValores nulos después de la imputación: {df_clean.isnull().sum().sum()}")

### 4.2 Tratamiento de Outliers

In [ ]:
# Detección de outliers usando IQR
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

print("Detección de outliers:")
for col in numeric_cols:
    outliers, lower, upper = detect_outliers_iqr(df_clean, col)
    print(f"{col}: {len(outliers)} outliers detectados (rango: [{lower:.2f}, {upper:.2f}])")

# NOTA: Decide si eliminar o transformar outliers según el contexto
# df_clean = df_clean[...] # Filtrar outliers si es necesario

### 4.3 Codificación de Variables Categóricas

In [ ]:
# Identificar variables categóricas
categorical_cols = df_clean.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Variables categóricas: {categorical_cols}")

# Codificación (ajustar según necesidad)
# One-Hot Encoding para variables con pocas categorías
# Label Encoding para variables ordinales

if categorical_cols:
    # Ejemplo: One-Hot Encoding
    df_encoded = pd.get_dummies(df_clean, columns=categorical_cols, drop_first=True)
    print(f"\nDataset después de encoding: {df_encoded.shape}")
else:
    df_encoded = df_clean.copy()
    print("\nNo hay variables categóricas para codificar")

### 4.4 Normalización/Estandarización

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Separar características y variable objetivo
X = df_encoded.drop(target_col, axis=1)
y = df_encoded[target_col]

# Estandarización (media=0, std=1)
scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X),
    columns=X.columns,
    index=X.index
)

print(f"Características estandarizadas: {X_scaled.shape}")
print(f"\nEstadísticas después de estandarización:")
display(X_scaled.describe())

### 4.5 División Train/Test

In [ ]:
from sklearn.model_selection import train_test_split

# División 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"Conjunto de entrenamiento: {X_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}")
print(f"\nDistribución en entrenamiento:")
print(y_train.value_counts())
print(f"\nDistribución en prueba:")
print(y_test.value_counts())

## 5. Modelización Predictiva

### 5.1 Definición de Modelos

Implementaremos diferentes técnicas de Machine Learning:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# Diccionario de modelos
models = {
    'Logistic Regression': LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_STATE),
    'Random Forest': RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=RANDOM_STATE, n_estimators=100),
    'SVM': SVC(random_state=RANDOM_STATE, probability=True),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Naive Bayes': GaussianNB()
}

print(f"Modelos definidos: {len(models)}")
for name in models.keys():
    print(f"  - {name}")

### 5.2 Entrenamiento de Modelos

In [ ]:
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
import time

# Entrenar y evaluar cada modelo
results = {}
trained_models = {}

for name, model in tqdm(models.items(), desc="Entrenando modelos"):
    start_time = time.time()
    
    # Entrenamiento
    model.fit(X_train, y_train)
    
    # Validación cruzada
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
    # Predicciones
    y_pred = model.predict(X_test)
    
    # Tiempo de entrenamiento
    training_time = time.time() - start_time
    
    # Guardar resultados
    results[name] = {
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'predictions': y_pred,
        'training_time': training_time
    }
    trained_models[name] = model
    
    print(f"{name}: CV Accuracy = {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

### 5.3 Comparación Inicial de Modelos

In [ ]:
# Crear DataFrame con resultados
results_df = pd.DataFrame({
    'Model': list(results.keys()),
    'CV Mean Accuracy': [results[m]['cv_mean'] for m in results],
    'CV Std': [results[m]['cv_std'] for m in results],
    'Training Time (s)': [results[m]['training_time'] for m in results]
}).sort_values('CV Mean Accuracy', ascending=False)

display(results_df)

# Visualizar comparación
plt.figure(figsize=(12, 6))
plt.bar(results_df['Model'], results_df['CV Mean Accuracy'], 
        yerr=results_df['CV Std'], capsize=5, alpha=0.7)
plt.xlabel('Modelo')
plt.ylabel('Accuracy (Validación Cruzada)')
plt.title('Comparación de Modelos - Validación Cruzada')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../results/visualizations/model_comparison_cv.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Evaluación de Modelos

### 6.1 Métricas de Evaluación

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

# Calcular métricas para cada modelo
evaluation_results = []

for name in results.keys():
    y_pred = results[name]['predictions']
    
    # Calcular métricas (ajustar average según el problema)
    metrics = {
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred, average='weighted')
    }
    evaluation_results.append(metrics)

evaluation_df = pd.DataFrame(evaluation_results).sort_values('F1-Score', ascending=False)
display(evaluation_df)

# Guardar resultados
evaluation_df.to_csv('../results/metrics/model_evaluation.csv', index=False)
print("\nResultados guardados en: results/metrics/model_evaluation.csv")

### 6.2 Matrices de Confusión

In [ ]:
# Seleccionar los 3 mejores modelos
top_models = evaluation_df.head(3)['Model'].tolist()

fig, axes = plt.subplots(1, len(top_models), figsize=(15, 4))

for idx, model_name in enumerate(top_models):
    y_pred = results[model_name]['predictions']
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                ax=axes[idx], cbar=False)
    axes[idx].set_title(f'{model_name}\nAccuracy: {evaluation_df[evaluation_df["Model"]==model_name]["Accuracy"].values[0]:.4f}')
    axes[idx].set_xlabel('Predicción')
    axes[idx].set_ylabel('Real')

plt.tight_layout()
plt.savefig('../results/visualizations/confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

### 6.3 Reportes de Clasificación

In [ ]:
# Reporte detallado del mejor modelo
best_model_name = evaluation_df.iloc[0]['Model']
y_pred_best = results[best_model_name]['predictions']

print(f"Reporte de clasificación - {best_model_name}")
print("=" * 50)
print(classification_report(y_test, y_pred_best))

### 6.4 Curvas ROC (para clasificación binaria/multiclase)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# Determinar número de clases
n_classes = len(np.unique(y))

if n_classes == 2:
    # Clasificación binaria
    plt.figure(figsize=(10, 8))
    
    for model_name in top_models:
        model = trained_models[model_name]
        if hasattr(model, 'predict_proba'):
            y_score = model.predict_proba(X_test)[:, 1]
            fpr, tpr, _ = roc_curve(y_test, y_score)
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Curvas ROC - Comparación de Modelos')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.savefig('../results/visualizations/roc_curves.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print(f"Clasificación multiclase ({n_classes} clases). Usar estrategia one-vs-rest para ROC.")

### 6.5 Importancia de Características

In [ ]:
# Para modelos basados en árboles
tree_based_models = ['Random Forest', 'Gradient Boosting', 'Decision Tree']

for model_name in tree_based_models:
    if model_name in trained_models:
        model = trained_models[model_name]
        if hasattr(model, 'feature_importances_'):
            # Obtener importancias
            importances = model.feature_importances_
            feature_importance_df = pd.DataFrame({
                'Feature': X_train.columns,
                'Importance': importances
            }).sort_values('Importance', ascending=False)
            
            # Visualizar top 15
            plt.figure(figsize=(10, 6))
            top_features = feature_importance_df.head(15)
            plt.barh(top_features['Feature'], top_features['Importance'])
            plt.xlabel('Importancia')
            plt.title(f'Top 15 Características - {model_name}')
            plt.gca().invert_yaxis()
            plt.tight_layout()
            plt.savefig(f'../results/visualizations/feature_importance_{model_name.replace(" ", "_").lower()}.png', 
                       dpi=300, bbox_inches='tight')
            plt.show()
            
            print(f"\nTop 10 características - {model_name}:")
            display(feature_importance_df.head(10))
            break  # Mostrar solo uno para no saturar

### 6.6 Optimización de Hiperparámetros (Opcional)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Optimizar el mejor modelo
best_model_name = evaluation_df.iloc[0]['Model']

# Ejemplo con Random Forest
if best_model_name == 'Random Forest':
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    print(f"Optimizando hiperparámetros para {best_model_name}...")
    print("NOTA: Esto puede tomar varios minutos")
    
    grid_search = GridSearchCV(
        RandomForestClassifier(random_state=RANDOM_STATE),
        param_grid,
        cv=5,
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )
    
    # Descomentar para ejecutar (puede ser lento)
    # grid_search.fit(X_train, y_train)
    # print(f"\nMejores parámetros: {grid_search.best_params_}")
    # print(f"Mejor score: {grid_search.best_score_:.4f}")
    
    print("\n⚠️ Optimización comentada para ahorrar tiempo. Descomentar si es necesario.")

## 7. Discusión de Resultados

### 7.1 Resumen de Resultados

In [ ]:
print("=" * 60)
print("RESUMEN DE RESULTADOS")
print("=" * 60)

print(f"\nDataset: {df.shape[0]} instancias, {df.shape[1]} características")
print(f"División: {len(X_train)} entrenamiento, {len(X_test)} prueba")
print(f"\nModelos evaluados: {len(models)}")

print("\n" + "=" * 60)
print("TOP 3 MODELOS")
print("=" * 60)
display(evaluation_df.head(3))

best_model = evaluation_df.iloc[0]
print(f"\n🏆 MEJOR MODELO: {best_model['Model']}")
print(f"   - Accuracy: {best_model['Accuracy']:.4f}")
print(f"   - Precision: {best_model['Precision']:.4f}")
print(f"   - Recall: {best_model['Recall']:.4f}")
print(f"   - F1-Score: {best_model['F1-Score']:.4f}")

### 7.2 Análisis Crítico

**Fortalezas del análisis:**
- [Menciona los aspectos positivos]
- [Qué funcionó bien]
- [Resultados destacables]

**Limitaciones:**
- [Limitaciones del dataset]
- [Limitaciones de los modelos]
- [Posibles sesgos]

**Posibles mejoras:**
- [Técnicas adicionales a probar]
- [Feature engineering adicional]
- [Optimización de hiperparámetros]
- [Ensemble methods]

### 7.3 Interpretación de Resultados

[Discute qué significan los resultados en el contexto del problema original]

**Preguntas clave a responder:**
1. ¿Los modelos tienen un rendimiento aceptable para el problema?
2. ¿Qué características son más importantes para la predicción?
3. ¿Hay evidencia de overfitting o underfitting?
4. ¿Los resultados son consistentes con el conocimiento del dominio?
5. ¿El modelo es interpretable y explicable?
6. ¿Cuáles son las implicaciones prácticas de los resultados?

## 8. Conclusiones

### 8.1 Logros Principales

- [Conclusión 1]
- [Conclusión 2]
- [Conclusión 3]

### 8.2 Trabajo Futuro

- [Línea de investigación 1]
- [Línea de investigación 2]
- [Mejoras propuestas]

### 8.3 Lecciones Aprendidas

- [Lección 1]
- [Lección 2]
- [Lección 3]

---

## Referencias

1. [Referencia del dataset]
2. [Documentación de librerías utilizadas]
3. [Papers o recursos relevantes]

---

## Notas Finales

**Para compartir en Kaggle:**
1. Exporta este notebook
2. Incluye el dataset (si es posible)
3. Añade una descripción clara del problema
4. Documenta los resultados

**Para el informe (máximo 20 páginas):**
- Utiliza las visualizaciones generadas
- Resume las tablas de resultados
- Enfócate en la interpretación y discusión
- Anexa código relevante si es necesario

**Para el video (5 minutos):**
- 0:00-0:30: Introducción y contexto
- 0:30-1:30: Dataset y análisis descriptivo
- 1:30-3:00: Modelos y metodología
- 3:00-4:30: Resultados principales
- 4:30-5:00: Conclusiones y trabajo futuro